In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as sps
import plotly.graph_objs as go

# Точечные оценки

In [ ]:
n1 = 10
n2 = 1000
n_exp = 10

In [ ]:
mean_1 = []
mean_2 = []
std_1 = []
std_2 = []
for _ in range(n_exp):
    x1 = sps.norm.rvs(loc=1, scale=1, size=n1)
    mean_1.append(x1.mean())
    std_1.append(x1.std(ddof=1))
    x2 = sps.norm.rvs(loc=1, scale=1, size=n2)
    mean_2.append(x2.mean())
    std_2.append(x2.std(ddof=1))

In [ ]:
mean_1

In [ ]:
mean_2

In [ ]:
std_1

In [ ]:
std_2

# Доверительные интервал

In [ ]:
alpha = 0.05

n1 = 10
n2 = 1000
n_exp = 10000

In [ ]:
# коэффициент
c = sps.norm.ppf(1 - alpha/2)

In [ ]:
c

In [ ]:
mean_1 = []
mean_2 = []
std_1 = []
std_2 = []
for _ in range(n_exp):
    x1 = sps.norm.rvs(loc=1, scale=1, size=n1)
    mean_1.append(x1.mean())
    std_1.append(x1.std(ddof=1))
    x2 = sps.norm.rvs(loc=1, scale=1, size=n2)
    mean_2.append(x2.mean())
    std_2.append(x2.std(ddof=1))
    
mean_1 = np.array(mean_1)
mean_2 = np.array(mean_2)
std_1 = np.array(std_1)
std_2 = np.array(std_2)

In [ ]:
CI_1 = np.array(list(zip(mean_1 - c*std_1/np.sqrt(n1), mean_1 + c*std_1/np.sqrt(n1))))

In [ ]:
CI_2 = np.array(list(zip(mean_2 - c*std_2/np.sqrt(n2), mean_2 + c*std_2/np.sqrt(n2))))

In [ ]:
CI_1

In [ ]:
CI_2

In [ ]:
df_ci_1 = pd.DataFrame(CI_1, columns=["left", "right"])
df_ci_1["contain_real_mean"] = 1
df_ci_1.loc[(1 < df_ci_1["left"]) | (1 > df_ci_1["right"]), "contain_real_mean"] = 0
df_ci_1["contain_real_mean"].sum() / df_ci_1["contain_real_mean"].shape[0] 

In [ ]:
df_ci_2 = pd.DataFrame(CI_2, columns=["left", "right"])
df_ci_2["contain_real_mean"] = 1
df_ci_2.loc[(1 < df_ci_2["left"]) | (1 > df_ci_2["right"]), "contain_real_mean"] = 0
df_ci_2["contain_real_mean"].sum() / df_ci_2["contain_real_mean"].shape[0] 

### Отрисовка

In [ ]:
size = 1000

x1 = sps.norm.rvs(loc=1, scale=11.5, size=size)
mean_1 = x1.mean()
std_1 = x1.std(ddof=1)

x2 = sps.norm.rvs(loc=1.3, scale=1, size=size)
mean_2 = x2.mean()
std_2 = x2.std(ddof=1)
    
CI_1 = [mean_1 - c*std_1 / np.sqrt(size), mean_1 + c*std_1 / np.sqrt(size)]

CI_2 = [mean_2 - c*std_2 / np.sqrt(size), mean_2 + c*std_2 / np.sqrt(size)]

In [ ]:
CI_1

In [ ]:
CI_2

In [ ]:
fig = go.Figure()
fig.add_shape(type="rect",
    x0=CI_1[0], y0=1, x1=CI_1[1], y1=2,
    fillcolor="lavender",
)
fig.add_shape(type="rect",
    x0=CI_2[0], y0=0, x1=CI_2[1], y1=1,
    fillcolor="RoyalBlue",
)
fig.update_shapes(dict(xref='x', yref='y'))
fig.show()

## Уилсон

In [ ]:
size = 1000

x1 = sps.bernoulli.rvs(p=0.015, size=size)
mean_1 = x1.mean()
std_1 = x1.std(ddof=1)

x2 = sps.bernoulli.rvs(p=0.025, size=size)
mean_2 = x2.mean()
std_2 = x2.std(ddof=1)
    
CI_1 = [mean_1 - c*std_1 / np.sqrt(size), mean_1 + c*std_1 / np.sqrt(size)]

CI_2 = [mean_2 - c*std_2 / np.sqrt(size), mean_2 + c*std_2 / np.sqrt(size)]

In [ ]:
CI_1, CI_2

In [ ]:
z = sps.norm.ppf(1 - alpha/2)

In [ ]:
CI_1_U = [(mean_1 + z**2 / (2*size) - z*np.sqrt(mean_1*(1-mean_1)/size + z**2/(4*size**2))) / (1 + z**2 / size),
          (mean_1 + z**2 / (2*size) + z*np.sqrt(mean_1*(1-mean_1)/size + z**2/(4*size**2))) / (1 + z**2 / size)]

In [ ]:
CI_2_U = [(mean_2 + z**2 / (2*size) - z*np.sqrt(mean_2*(1-mean_2)/size + z**2/(4*size**2))) / (1 + z**2 / size),
          (mean_2 + z**2 / (2*size) + z*np.sqrt(mean_2*(1-mean_2)/size + z**2/(4*size**2))) / (1 + z**2 / size)]

In [ ]:
CI_1_U, CI_2_U

# Bootstrap

In [ ]:
def get_bootstrap_sample_indices(sample_size: int, n_samples: int) -> np.ndarray:
    return np.random.randint(0, sample_size, (n_samples, sample_size))

def get_bootstrap_samples(X: np.ndarray, n_samples: int) -> np.ndarray:
    return X[get_bootstrap_sample_indices(len(X), n_samples)]

In [ ]:
size = 100
x_a = sps.expon.rvs(loc=4, scale=16, size=size)

In [ ]:
np.median(x_a)

In [ ]:
X_bootstrap = get_bootstrap_samples(x_a, n_samples=10_000)

In [ ]:
X_bootstrap

In [ ]:
metrics_mean = np.median(X_bootstrap, axis=1)

In [ ]:
go.Figure([go.Histogram(x=metrics_mean)])

In [ ]:
alpha = 0.05

In [ ]:
CI = np.percentile(metrics_mean, (alpha * 100, (1 - alpha) * 100))

In [ ]:
CI

# T-test

In [ ]:
size = 30
n_sample = 1000
p_values = []
for _ in range(n_sample):
    x_a = sps.norm.rvs(loc=1, scale=11.5, size=size)
    x_b = sps.norm.rvs(loc=1, scale=1, size=size)
    p_values.append(sps.mannwhitneyu(x_a, x_b).pvalue)
p_values = np.array(p_values)

In [ ]:
p_values[p_values < 0.05].shape[0] / n_sample

In [ ]:
size = 30
n_sample = 1000
p_values = []
for _ in range(n_sample):
    x_a = sps.expon.rvs(loc=1, scale=1, size=size)
    x_b = sps.expon.rvs(loc=1, scale=1, size=size)
    p_values.append(sps.ttest_ind(x_a, x_b, equal_var=False).pvalue)
p_values = np.array(p_values)

In [ ]:
p_values[p_values < 0.05].shape[0] / n_sample

# Манн-Уитни

In [ ]:
x_a = [15, 12, 12, 10]
x_b = [14, 11, 9, 9]

In [ ]:
sps.mannwhitneyu(x_a, x_b)

In [ ]:
xr_a = [0, 2.5, 2.5, 5]
xr_b = [1, 4, 6.5, 6.5]

In [ ]:
sps.ttest_ind(xr_a, xr_b)

In [ ]:
size = 30
n_sample = 1000
p_values = []
for _ in range(n_sample):
    x_a = sps.norm.rvs(loc=1, scale=1, size=size)
    x_b = sps.norm.rvs(loc=1, scale=1, size=size)
    p_values.append(sps.mannwhitneyu(x_a, x_b).pvalue)
p_values = np.array(p_values)

In [ ]:
p_values[p_values < 0.05].shape[0] / n_sample

In [ ]:
size = 30
n_sample = 1000
p_values = []
for _ in range(n_sample):
    x_a = sps.expon.rvs(loc=1, scale=1, size=size)
    x_b = sps.expon.rvs(loc=1, scale=1, size=size)
    p_values.append(sps.mannwhitneyu(x_a, x_b).pvalue)
p_values = np.array(p_values)

In [ ]:
p_values[p_values < 0.05].shape[0] / n_sample

# Мощность критерия

In [ ]:
size = 100
effect = 10
n_exp = 10_000
alpha = 0.05

p_values = []
for i in range(n_exp):
    x_a = sps.norm.rvs(loc=10, scale=3, size=size)
    x_b = sps.norm.rvs(loc=10 + effect, scale=3, size=size)
    p_value = sps.ttest_ind(x_a, x_b).pvalue
    p_values.append(p_value)
p_values = np.array(p_values)

Оценка мощности

In [ ]:
p_values[p_values < alpha].shape[0] / p_values.shape[0] * 100

# Корректность критерия

Вариант с нормальными распределениями

In [ ]:
p_values = []
size = 5
n_exp = 1000
for i in range(n_exp):
    x_a = sps.norm.rvs(loc=1, scale=1, size=size)
    x_b = sps.norm.rvs(loc=1, scale=1, size=size)
    p_value = sps.ttest_ind(x_a, x_b, equal_var=False).pvalue
    p_values.append(p_value)
p_values = np.array(p_values)

Вариант с смесью нормальных распределений

In [ ]:
p_values = []
size = 100
n_exp = 10000
for i in range(n_exp):
    x_a = np.append(
        sps.norm.rvs(loc=1, scale=1, size=size),
        sps.norm.rvs(loc=0.5, scale=0.1, size=size),
    )
    x_b = np.append(
        sps.norm.rvs(loc=1, scale=1, size=size),
        sps.norm.rvs(loc=0.5, scale=0.1, size=size),
    )
    p_value = sps.ttest_ind(x_a, x_b).pvalue
    p_values.append(p_value)
p_values = np.array(p_values)

Другой вариант со смесью нормальных распеределений

In [ ]:
p_values = []
size = 100
n_exp = 10000
for i in range(n_exp):
    a_1 = sps.norm.rvs(loc=1, scale=1, size=size)
    a_2 = sps.norm.rvs(loc=0.5, scale=0.1, size=size)
    a_mask = sps.randint.rvs(0, 2, size=size).astype(bool)
    x_a = np.append(
        a_1[a_mask],
        a_2[~a_mask],
    )
    
    b_1 = sps.norm.rvs(loc=1, scale=1, size=size)
    b_2 = sps.norm.rvs(loc=0.5, scale=0.1, size=size)
    b_mask = sps.randint.rvs(0, 2, size=size).astype(bool)
    x_b = np.append(
        b_1[b_mask],
        b_2[~b_mask],
    )
    p_value = sps.ttest_ind(x_a, x_b, equal_var=True).pvalue
    p_values.append(p_value)
p_values = np.array(p_values)

Экспоненциальные распределения

In [ ]:
p_values = []
size = 15
n_exp = 10000
for i in range(n_exp):
    x_a = sps.expon.rvs(scale=1/2, size=size)
    x_b = sps.expon.rvs(scale=1/2, size=size)
    p_value = sps.ttest_ind(x_a, x_b).pvalue
    p_values.append(p_value)
p_values = np.array(p_values)

Бернулли распределение

In [ ]:
p_values = []
size = 1000
n_exp = 10000
for i in range(n_exp):
    x_a = sps.bernoulli.rvs(p=0.05, size=size)
    x_b = sps.bernoulli.rvs(p=0.05, size=size)
    p_value = sps.ttest_ind(x_a, x_b).pvalue
    p_values.append(p_value)
p_values = np.array(p_values)

### Оценка распределения

In [ ]:
fig = go.Figure([go.Histogram(x=x_a, opacity=0.8),
                 go.Histogram(x=x_b, opacity=0.8)])
fig.update_layout(barmode="overlay")

### Оценка критерия на данных

In [ ]:
alpha = 0.05

In [ ]:
p_values[p_values < alpha].shape[0] / p_values.shape[0]

In [ ]:
probs = []
x = [0.01 * i for i in range(101)]
for i in range(101):
    alpha_step = 0.01 * i
    probs.append(p_values[p_values < alpha_step].shape[0] / p_values.shape[0])

In [ ]:
fig = go.Figure([go.Scatter(x=x, y=probs, mode="markers", name="p_value"),
                 go.Scatter(x=x, y=x, mode="lines", name="uniform")])
fig.update_layout(height=600, width=600, title="Q-Q plot")

In [ ]:
fig = go.Figure([go.Histogram(x=p_values, xbins={"start":0, "end":1, "size": 0.1})])
fig